In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

from IPython.display import display
from IPython.display import Image

sys.path.append('/project/lgrandi/dbarge/')

print()
print("-> Python Executable (sys.executable):   " + sys.executable)
print("-> Python Path (sys.path):")
for x in sys.path: print("   " + x)
print()


####################################################################################################
####################################################################################################

import pax
print("-> Pax version: " + pax.__version__)


####################################################################################################
####################################################################################################


from pax_utils import s1s2_utils
from pax_utils.helpers import *


####################################################################################################
####################################################################################################

import tensorflow as tf

print ("-> Tensorflow version:   " + tf.__version__ + "\n")

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model


####################################################################################################
####################################################################################################

from models import *


####################################################################################################
####################################################################################################

v_drift = 1.44e-4

print("Ready")



-> Python Executable (sys.executable):   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/bin/python
-> Python Path (sys.path):
   
   /project/lgrandi/dbarge/pax_v6.5.1
   /home/dbarge/reconstruction/z_s2waveforms
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python36.zip
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/lib-dynload
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/IPython/extensions
   /home/dbarge/.ipython
   /project/lgrandi/dbarge/

-> Pax version: 6.5.1
-> Tensorflow version:   1.10.0



Using TensorFlow backend.


Ready


In [2]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = '/project/lgrandi/dbarge/simulation/wimp/pax_v6.8.3/merged/merged_all_200000.pkl'

df_all   = pd.read_pickle(file_all)
df_all   = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[0:nEventsTrain][:]

df_all['intr_depth'] = v_drift*df_all['intr_drift_time'] 


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['depth']]
df_train_input = df_all[:][['intr_depth']]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (100000, 1)
Truth shape: (100000, 1)

Input DataFrame: 


,intr_depth
0,57.83616
1,58.27248
2,34.41168
3,64.75680
4,51.72192



Truth DataFrame:


,depth
0,57.456815
1,57.780505
2,34.074937
3,64.494053
4,51.510242


In [40]:

##########################################################################################
##########################################################################################

lst_models = [
    bargeModel_z()
]



In [42]:

##########################################################################################
##########################################################################################

for iModel in range(0, len(lst_models)):
    
    ######################################################################################
    ######################################################################################

    model, model_name = lst_models[iModel]
    
    
    ######################################################################################
    ######################################################################################

    epochs     = 10
    batch_size = 64
    verbose    = 1
    
    
    ######################################################################################
    # Fit Model
    ######################################################################################
    
    print()
    #print("Model Summary:")
    #model.summary()
    print()
    model.fit(df_train_input, df_train_truth, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print("Training Complete!")

        
    ######################################################################################
    # Save Model
    ######################################################################################

    folder   = "models" + "/"    
    name     = 'model' + '_' + model_name + '_' + 'elu' #model.activation
    name_h5  = folder + name + ".h5"
    name_png = folder + name + ".png"

    model.save(name_h5, overwrite=True)
        
   
    ######################################################################################
    ######################################################################################

    plot_model(model, to_file=name_png, show_layer_names=True, show_shapes=True)
    
    Image(filename=name_png) 
    

    ######################################################################################
    ######################################################################################
    
    continue
    
    



Epoch 1/10
100000/100000 [==============================] - 4s 38us/step - loss: 0.6496
Epoch 2/10
100000/100000 [==============================] - 4s 39us/step - loss: 0.6507
Epoch 3/10
100000/100000 [==============================] - 4s 38us/step - loss: 0.6737
Epoch 4/10
100000/100000 [==============================] - 4s 38us/step - loss: 0.6713
Epoch 5/10
100000/100000 [==============================] - 4s 38us/step - loss: 0.6470
Epoch 6/10
100000/100000 [==============================] - 4s 41us/step - loss: 0.6368
Epoch 7/10
100000/100000 [==============================] - 4s 40us/step - loss: 0.6217
Epoch 8/10
100000/100000 [==============================] - 4s 40us/step - loss: 0.6227
Epoch 9/10
100000/100000 [==============================] - 4s 40us/step - loss: 0.6062
Epoch 10/10
100000/100000 [==============================] - 4s 39us/step - loss: 0.6047
Training Complete!
